# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
!pip install pandas

In [7]:
import pandas as pd
df = pd.read_csv(r"C:\Users\jesus\lab-chains-in-langchain\data\Data.csv")

In [9]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [12]:
!pip install langchain_openai

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [16]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [18]:
# Step 2: Create the prompt template with a string template
prompt = ChatPromptTemplate.from_template(
    "Write a detailed description for the following product: {product_name}. Include its features, benefits, and potential uses."
)

In [20]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\jesus\AppData\Local\Temp\ipykernel_23268\1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [22]:
# Step 4: Select a product type to describe
product = {"product_name": "smartphone"}

# Step 5: Run the chain and get the product description
response = chain.run(product)

# Step 6: Print the response
print(response)

C:\Users\jesus\AppData\Local\Temp\ipykernel_23268\1771342572.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(product)


A smartphone is a multifunctional device that combines the features of a mobile phone with those of a computer. It typically has a touchscreen interface, allowing users to easily navigate through various applications and functions. Smartphones are equipped with a wide range of features that make them incredibly versatile and convenient for everyday use.

Some common features of smartphones include:

1. Connectivity: Smartphones can connect to the internet through Wi-Fi or cellular data, allowing users to browse the web, access social media, and send emails.

2. Camera: Most smartphones come equipped with high-quality cameras that allow users to take photos and videos on the go.

3. Apps: Smartphones support a wide range of applications that can be downloaded from app stores, allowing users to customize their device with tools for productivity, entertainment, communication, and more.

4. Storage: Smartphones typically have built-in storage space for photos, videos, music, and other file

## SimpleSequentialChain

In [25]:
from langchain.chains import SimpleSequentialChain

In [27]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a detailed description for the following product: {product_name}. Include its features, benefits, and potential uses."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Now summarize the following product description: {description}. Highlight the key selling points and features."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [31]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [33]:
# Select a product to describe
product = {"product_name": "smartphone"}

# Make sure to pass the 'input' key explicitly, as SimpleSequentialChain expects it
response = overall_simple_chain.run({"input": product})

# Print the final result
print(response)



> Entering new SimpleSequentialChain chain...
A smartphone is a handheld device that combines the functionality of a mobile phone with various other features and capabilities, such as internet browsing, email, music playback, camera, and apps. 

One of the key features of a smartphone is its ability to connect to the internet, allowing users to stay connected and access a wide range of online services and content. This includes email, social media platforms, news websites, and online shopping.

Smartphones also come with a variety of apps that can be downloaded and installed to enhance functionality. This includes productivity apps for tasks like calendars and note-taking, entertainment apps for streaming music and videos, and gaming apps for entertainment.

The camera feature of a smartphone allows users to take high-quality photos and videos on-the-go, making it convenient for capturing memories and sharing them with others. Many smartphones also come with advanced camera features 

**Repeat the above twice for different products**

## SequentialChain

In [35]:
from langchain.chains import SequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [37]:
# Initialize LLM
llm = ChatOpenAI(temperature=0.9)

# First prompt template: Translate the review (for example, translating to Spanish)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into Spanish: {review}"
)

# Chain 1: Translate the review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="translated_review")

In [39]:
# Second prompt template: Summarize the review
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in a few sentences: {translated_review}"
)

# Chain 2: Summarize the translated review
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [41]:
# Third prompt template: Translate the summary back to English
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following text back into English: {summary}"
)

# Chain 3: Translate the summary back to English
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="english_summary")

In [43]:
# Fourth prompt template: Generate a follow-up message based on both the translated review and the English summary
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the translated review: {translated_review} and the English summary: {english_summary}, create a follow-up message asking for more details."
)

# Chain 4: Generate a follow-up message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="follow_up_message")

In [45]:
# Create a new prompt to combine the results into a single final output
combine_prompt = ChatPromptTemplate.from_template(
    """
    Combine the following information into a single output:
    1. Translated Review: {translated_review}
    2. Summary: {summary}
    3. English Summary: {english_summary}
    4. Follow-Up Message: {follow_up_message}

    Provide all the information in a single output as a dictionary or a structured format.
    """
)

# Final chain to combine all outputs
combine_chain = LLMChain(llm=llm, prompt=combine_prompt, output_key="final_output")

In [47]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
# SequentialChain: Combine all the chains into one
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, combine_chain],
    input_variables=["review"],  # Input is the review itself
    output_variables=["final_output"],  # Single final output
    verbose=True
)

In [49]:
# Run the overall chain for the given review
response = overall_chain.run({"review": review})

# Print the raw response to inspect its structure
print("Raw response:", response)

# Check if response is a string or a dictionary and handle accordingly
if isinstance(response, dict):
    print(response["final_output"])  # If it's a dictionary, access "final_output"
else:
    print("Response is not a dictionary, it's a string. Here is the full response:")
    print(response)  # Print the full string response to see its content

NameError: name 'review' is not defined

**Repeat the above twice for different products or reviews**

## Router Chain

In [51]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [53]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [55]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [57]:
llm = ChatOpenAI(temperature=0)

In [59]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [61]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [63]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [65]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [73]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\jesus\AppData\Local\Temp\ipykernel_23268\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [75]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [77]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [79]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**